# Week 2 Assignment – Structured and Unstructured Data with Spark

**Course:** CSCE 5300 – Introduction to Big Data and Data Science  
**Topic:** Structured and Unstructured Data Processing  
**Tools:** PySpark, JSON/CSV parsing, DataFrame transformations

In this assignment, you will build upon your Spark installation from Week 1 to load and process both structured (CSV) and unstructured (JSON) data using Spark.

In [4]:
# Step 1: Setup Spark Environment
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, to_timestamp, hour, dayofweek

try:
    spark = SparkSession.builder \
        .appName("Week 2 Data Processing") \
        .getOrCreate()
    print("✅ Spark session created successfully!")
except Exception as e:
    print("❌ Failed to create Spark session:", e)

✅ Spark session created successfully!


## Part A – Structured Data (CSV)

We'll work with the Titanic dataset to explore structured data using Spark.

In [5]:
local_path = "data/titanic.csv"  # Example path — change if needed

df_csv = spark.read.option("header", "true").csv(local_path)

# Print schema and preview the data
df_csv.printSchema()
df_csv.show(5)

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17

In [6]:
# Drop null values and count survivors by gender
df_cleaned = df_csv.dropna(subset=["Survived", "Sex"])
df_cleaned.groupBy("Sex", "Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|       0|   81|
|female|       1|  233|
|  male|       1|  109|
|  male|       0|  468|
+------+--------+-----+



## Part B – Unstructured Data (JSON)

Now let’s work with Yelp Check-in data as unstructured input (semi-structured JSON).

In [9]:
# Load Yelp Check-in JSON (place your file in the same folder or update the path)
df_json = spark.read.json("data/yelp-dataset/yelp_academic_dataset_checkin.json")
df_json.printSchema()
df_json.show(5, truncate=False)

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|business_id           |date                                                                                                                                                                                                                                                                                                                             

In [10]:
# Split and transform the date field
df_split = df_json.withColumn("checkin_time", explode(split(df_json["date"], ", ")))
df_clean = df_split.withColumn("checkin_time", to_timestamp("checkin_time"))
df_clean.select("business_id", "checkin_time").show(5)

+--------------------+-------------------+
|         business_id|       checkin_time|
+--------------------+-------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:56|
|---kPU91CF4Lq2-Wl...|2020-06-02 22:18:06|
|---kPU91CF4Lq2-Wl...|2020-07-24 22:42:27|
|---kPU91CF4Lq2-Wl...|2020-10-24 21:36:13|
|---kPU91CF4Lq2-Wl...|2020-12-09 21:23:33|
+--------------------+-------------------+
only showing top 5 rows


In [12]:
# Extract time-based features and aggregate
df_features = df_clean.withColumn("hour", hour("checkin_time")) \
                      .withColumn("day_of_week", dayofweek("checkin_time"))
df_features.groupBy("hour").count().orderBy("hour").show()
df_features.groupBy("day_of_week").count().orderBy("day_of_week").show()

+----+-------+
|hour|  count|
+----+-------+
|   0|1155092|
|   1| 935985|
|   2| 665907|
|   3| 440702|
|   4| 264905|
|   5| 152476|
|   6|  85066|
|   7|  52295|
|   8|  35589|
|   9|  37079|
|  10|  63824|
|  11| 115876|
|  12| 201427|
|  13| 296364|
|  14| 407969|
|  15| 587904|
|  16| 873108|
|  17|1018438|
|  18| 995358|
|  19| 922177|
+----+-------+
only showing top 20 rows


[Stage 19:==============>                                          (3 + 9) / 12]

+-----------+-------+
|day_of_week|  count|
+-----------+-------+
|          1|2480701|
|          2|1491993|
|          3|1460432|
|          4|1541769|
|          5|1612496|
|          6|1959015|
|          7|2810469|
+-----------+-------+



## Part C – Spark Exploration Challenge 🔍

In this section, you will complete the following tasks by filling in the missing parts of the provided code:

### 🎯 Your Task:
- Show the **last 5 rows** of the Titanic dataset using Spark.
- Select only important columns such as: `PassengerId`, `Name`, `Survived`, `Pclass`, `Sex`, `Age`
- (Optional Bonus) Sort by `Age` in descending order and show the top 5 oldest passengers.

### 🧩 Starter Code:
Run the following cells and **fill in the blanks** where indicated.

In [13]:
# Show last 5 rows (Hint: sort by a column and use .tail equivalent)
# Replace '___' with the correct DataFrame methods

df_csv.orderBy("PassengerId", ascending=False).limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|         99|       1|     2|Doling, Mrs. John...|female|  34|    0|    1|  231919|     23|   NULL|       S|
|         98|       1|     1|Greenfield, Mr. W...|  male|  23|    0|    1|PC 17759|63.3583|D10 D12|       C|
|         97|       0|     1|Goldschmidt, Mr. ...|  male|  71|    0|    0|PC 17754|34.6542|     A5|       C|
|         96|       0|     3|Shorney, Mr. Char...|  male|NULL|    0|    0|  374910|   8.05|   NULL|       S|
|         95|       0|     3|   Coxon, Mr. Daniel|  male|  59|    0|    0|  364500|   7.25|   NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+



In [14]:
# Select important columns only (fill in the list)
columns_to_select = ["PassengerId", "Name" , "Survived", "Pclass", "Sex" , "Age"]

df_csv.select(columns_to_select).show(5)

+-----------+--------------------+--------+------+------+---+
|PassengerId|                Name|Survived|Pclass|   Sex|Age|
+-----------+--------------------+--------+------+------+---+
|          1|Braund, Mr. Owen ...|       0|     3|  male| 22|
|          2|Cumings, Mrs. Joh...|       1|     1|female| 38|
|          3|Heikkinen, Miss. ...|       1|     3|female| 26|
|          4|Futrelle, Mrs. Ja...|       1|     1|female| 35|
|          5|Allen, Mr. Willia...|       0|     3|  male| 35|
+-----------+--------------------+--------+------+------+---+
only showing top 5 rows


In [37]:
# BONUS: Show top 5 oldest passengers
df_csv.selectExpr("Name", "cast(Age as double) as Age").orderBy("Age", ascending=False).show(5)

+--------------------+----+
|                Name| Age|
+--------------------+----+
|Barkworth, Mr. Al...|80.0|
| Svensson, Mr. Johan|74.0|
|Artagaveytia, Mr....|71.0|
|Goldschmidt, Mr. ...|71.0|
|Connors, Mr. Patrick|70.5|
+--------------------+----+
only showing top 5 rows


### 🎯 Task 2: Yelp JSON
- Split the `date` field into individual timestamps
- Extract the **hour** from check-in times
- Group and count check-ins by hour

🔧 Fill in the blanks:

In [41]:
# Load JSON data (already done in earlier cell)
df_json = spark.read.json("data/yelp-dataset/yelp_academic_dataset_checkin.json")

# Transform the 'date' column into individual timestamps
from pyspark.sql.functions import split, explode, to_timestamp, hour

df_split = df_json.withColumn("checkin_time", explode(split(df_json["date"], ", ")))

# Convert to timestamp
df_clean = df_split.withColumn("checkin_time", to_timestamp("checkin_time"))

# Extract hour and group
df_hour = df_clean.withColumn("hour", hour("checkin_time"))
df_hour.groupBy("hour").count().orderBy("hour").show()

[Stage 48:======================================>                  (8 + 4) / 12]

+----+-------+
|hour|  count|
+----+-------+
|   0|1155092|
|   1| 935985|
|   2| 665907|
|   3| 440702|
|   4| 264905|
|   5| 152476|
|   6|  85066|
|   7|  52295|
|   8|  35589|
|   9|  37079|
|  10|  63824|
|  11| 115876|
|  12| 201427|
|  13| 296364|
|  14| 407969|
|  15| 587904|
|  16| 873108|
|  17|1018438|
|  18| 995358|
|  19| 922177|
+----+-------+
only showing top 20 rows


In [42]:
df_clean

DataFrame[business_id: string, date: string, checkin_time: timestamp]

In [44]:
# Group check-ins by business and count how many check-ins each business has
df_clean.groupBy("business_id").count().orderBy("count", ascending=False).show(5)

[Stage 54:>                                                       (0 + 12) / 12]

+--------------------+-----+
|         business_id|count|
+--------------------+-----+
|-QI8Qi8XWH3D8y8et...|52144|
|FEXhWNCMkv22qG04E...|40109|
|Eb1XmmLWyt_way5NN...|37562|
|c_4c5rJECZSfNgFj7...|37518|
|4i4kmYm9wgSNyF1b6...|31168|
+--------------------+-----+
only showing top 5 rows


In [45]:
from pyspark.sql.functions import max

# Find the most recent check-in time per business
df_clean.groupBy("business_id").agg(max("checkin_time").alias("latest_checkin")).orderBy("latest_checkin", ascending=False).show(5)

[Stage 57:>                                                       (0 + 12) / 12]

+--------------------+-------------------+
|         business_id|     latest_checkin|
+--------------------+-------------------+
|A2WOHcAk-BkMyf9Qg...|2022-01-19 16:48:37|
|CjNjiHsyV-gvtOsHZ...|2022-01-19 16:46:55|
|9m7mHDbSLIUjFwuuV...|2022-01-19 16:45:11|
|6R-hg8Ee51PDQsjaP...|2022-01-19 16:44:26|
|kZu9sbMkvbpDCpqmm...|2022-01-19 16:43:59|
+--------------------+-------------------+
only showing top 5 rows


## Part D – Reflection ✍️

**Write a short reflection (2–5 sentences):**
- What differences did you observe between handling CSV and JSON in Spark?
- What was easier or harder, and why?

_You can write this in the Markdown cell below or submit as a separate note._

### CSV vs JSON in Spark

I have previous experience with handling CSV files with pandas and so the data processing for CSV file with Spark felt familiar to me. Operations like groupBy(), count() were almost identical and operations like select(), orderBy() had familiarity to them. CSV files are well structured by design and that made it easier to manipulate and analyze.

The JSON data processing on the other hand felt a bit more tricky, owning to is semi-structured format. The JSON data needed some specific operations like explode() and split() to convert into individual records before I could start with the analysis steps. To me, it felt that I needed to put in more effort and needed to have deeper understanding of JSON structure to be able to even start working with this type of data.
